# LangGraph Memory Systems - Practical Exercises

## Learning Objectives

By the end of this notebook, you will be able to:
1. **Store and retrieve data** using LangGraph's InMemoryStore with namespaces and keys
2. **Build chatbots** with both short-term (checkpointer) and long-term (store) memory
3. **Define and use structured schemas** for memory using TypedDict and Pydantic
4. **Update complex schemas efficiently** using the Trustcall library
5. **Choose appropriate memory patterns** (profile vs collection) for different use cases
6. **Implement memory agents** that selectively update different types of memory

## Setup

In [ ]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph trustcall langchain_core

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

## Part 1: Memory Store Basics

### Exercise 1.1: Your First Memory Store

Let's start with the fundamentals. The LangGraph Memory Store provides a key-value storage system with namespaces (similar to directories) and keys (similar to filenames).

In [ ]:
import uuid
from langgraph.store.memory import InMemoryStore

# TODO: Create an InMemoryStore instance
store = # YOUR CODE HERE

# TODO: Create a namespace tuple for user "alice" and memory type "preferences"
namespace = # YOUR CODE HERE

# TODO: Store Alice's food preferences using put()
# Use a UUID for the key and a dictionary with "food": "Italian cuisine"
key = str(uuid.uuid4())
value = # YOUR CODE HERE
# YOUR CODE HERE to put the value

print(f"Stored preference with key: {key}")

In [ ]:
# TODO: Retrieve all memories from Alice's preferences namespace using search()
memories = # YOUR CODE HERE

# TODO: Print the first memory's key and value
print(f"Key: {memories[0].key}")
print(f"Value: {memories[0].value}")
print(f"Full metadata: {memories[0].dict()}")

In [ ]:
# TODO: Retrieve the specific memory using get() with namespace and key
specific_memory = # YOUR CODE HERE
print(f"Retrieved specific memory: {specific_memory.value}")

### Exercise 1.2: Building a Knowledge Base

Create a personal knowledge base that stores different types of information in organized namespaces.

In [ ]:
# TODO: Create a knowledge base for user "bob" with the following structure:
# - ("bob", "books"): Store 2 book recommendations
# - ("bob", "contacts"): Store 2 contact entries
# - ("bob", "notes"): Store 2 personal notes

knowledge_base = InMemoryStore()
user_id = "bob"

# Books namespace
books_namespace = # YOUR CODE HERE
book1 = {"title": "The Pragmatic Programmer", "author": "Hunt & Thomas", "rating": 5}
book2 = # YOUR CODE HERE - add another book

# YOUR CODE HERE - store both books

# Contacts namespace  
contacts_namespace = # YOUR CODE HERE
contact1 = {"name": "Alice Smith", "email": "alice@example.com", "relationship": "colleague"}
contact2 = # YOUR CODE HERE - add another contact

# YOUR CODE HERE - store both contacts

# Notes namespace
notes_namespace = # YOUR CODE HERE
note1 = {"topic": "Python Tips", "content": "Use list comprehensions for cleaner code"}
note2 = # YOUR CODE HERE - add another note

# YOUR CODE HERE - store both notes

In [ ]:
# TODO: Create a function to display all knowledge from a specific namespace
def display_knowledge_category(store, user_id, category):
    """Display all entries from a specific knowledge category"""
    namespace = # YOUR CODE HERE
    entries = # YOUR CODE HERE
    
    print(f"\n{category.upper()} for {user_id}:")
    print("-" * 30)
    for entry in entries:
        print(f"• {entry.value}")

# TODO: Display all categories
for category in ["books", "contacts", "notes"]:
    display_knowledge_category(knowledge_base, user_id, category)

## Part 2: Chatbot with Memory

### Exercise 2.1: Personal Assistant Chatbot

Build a chatbot that combines short-term (conversation history) and long-term (user facts) memory.

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.store.base import BaseStore
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables.config import RunnableConfig

# Initialize model
model = ChatOpenAI(model="gpt-4o", temperature=0)

# TODO: Complete the system message template
ASSISTANT_SYSTEM_MESSAGE = """
You are a helpful personal assistant with memory. 
You remember facts about your user to provide personalized assistance.

Here are the facts you remember about the user (may be empty):
{user_facts}

# YOUR INSTRUCTION HERE: Add instructions for how to use this memory
"""

# TODO: Complete the memory creation instruction
CREATE_MEMORY_INSTRUCTION = """
You are helping maintain a memory about the user.

Current facts: {existing_facts}

Instructions:
# YOUR INSTRUCTION HERE: Add detailed instructions for extracting user facts
"""

def personal_assistant(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Main assistant that uses memory to personalize responses"""
    
    # TODO: Get user_id from config
    user_id = # YOUR CODE HERE
    
    # TODO: Create namespace and retrieve user facts
    namespace = # YOUR CODE HERE
    memory_item = store.get(namespace, "user_facts")
    
    # TODO: Extract facts or use default message
    user_facts = # YOUR CODE HERE - handle both existing and empty memory
    
    # TODO: Create system message with user facts
    system_msg = # YOUR CODE HERE
    
    # TODO: Get response from model with system message and conversation
    response = # YOUR CODE HERE
    
    return {"messages": response}

def save_user_facts(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Extract and save facts about the user from conversation"""
    
    # TODO: Get user_id and namespace
    user_id = # YOUR CODE HERE
    namespace = # YOUR CODE HERE
    
    # TODO: Get existing facts
    existing_memory = store.get(namespace, "user_facts")
    existing_facts = # YOUR CODE HERE - handle existing vs empty
    
    # TODO: Create system message for fact extraction
    system_msg = # YOUR CODE HERE
    
    # TODO: Extract new facts from conversation
    new_facts = # YOUR CODE HERE
    
    # TODO: Store the updated facts
    # YOUR CODE HERE

print("Functions defined. Ready to build the graph!")

In [ ]:
# TODO: Build and compile the graph
builder = StateGraph(MessagesState)

# TODO: Add nodes
# YOUR CODE HERE

# TODO: Add edges
# YOUR CODE HERE

# TODO: Create store and checkpointer
long_term_memory = # YOUR CODE HERE
short_term_memory = # YOUR CODE HERE

# TODO: Compile graph
assistant_graph = # YOUR CODE HERE

print("Personal assistant graph compiled!")

In [ ]:
# Test the assistant
config = {"configurable": {"thread_id": "test_1", "user_id": "sarah"}}

# First conversation
messages = [HumanMessage(content="Hi! I'm Sarah. I'm a software engineer living in Seattle.")]
for chunk in assistant_graph.stream({"messages": messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# Continue conversation
messages = [HumanMessage(content="I love hiking and trying new coffee shops on weekends.")]
for chunk in assistant_graph.stream({"messages": messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# Test memory persistence in new thread
new_config = {"configurable": {"thread_id": "test_2", "user_id": "sarah"}}
messages = [HumanMessage(content="Can you recommend something fun for this weekend?")]

for chunk in assistant_graph.stream({"messages": messages}, new_config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

## Part 3: Structured Memory with Schemas

### Exercise 3.1: Customer Profile Schema

Create a structured customer service chatbot that maintains detailed customer profiles.

In [ ]:
from typing import TypedDict, List, Optional
from pydantic import BaseModel, Field

# TODO: Define a CustomerProfile using Pydantic BaseModel
class CustomerProfile(BaseModel):
    """Comprehensive customer profile for customer service"""
    # TODO: Add fields for:
    # - name: Optional[str]
    # - email: Optional[str] 
    # - phone: Optional[str]
    # - company: Optional[str]
    # - subscription_tier: Optional[str] ("basic", "premium", "enterprise")
    # - previous_issues: List[str] (past support issues)
    # - preferences: List[str] (communication preferences, product interests)
    # - satisfaction_rating: Optional[int] (1-10 scale)
    pass  # Replace with your implementation

# TODO: Test the schema
sample_profile = CustomerProfile(
    name="John Doe",
    email="john@company.com",
    subscription_tier="premium",
    previous_issues=["login problems", "billing question"],
    preferences=["email communication", "technical features"]
)

print(f"Sample profile: {sample_profile.model_dump()}")

In [ ]:
# TODO: Create a customer service chatbot using the schema
from langchain_core.messages import SystemMessage

# TODO: Use with_structured_output to create a model that outputs CustomerProfile
profile_model = # YOUR CODE HERE

# TODO: Test profile extraction
test_conversation = [
    HumanMessage("Hi, I'm Jane Smith from TechCorp. I'm having trouble with our enterprise account."),
    HumanMessage("My email is jane.smith@techcorp.com and this is the third time I've had login issues.")
]

# TODO: Extract profile using the structured model
extracted_profile = # YOUR CODE HERE
print(f"Extracted profile: {extracted_profile}")

### Exercise 3.2: Trustcall for Profile Updates

Learn to use Trustcall for efficient profile updates without regenerating the entire profile.

In [ ]:
from trustcall import create_extractor

# TODO: Create a Trustcall extractor for CustomerProfile
profile_extractor = # YOUR CODE HERE - use create_extractor with CustomerProfile

# Initial conversation to create profile
initial_conversation = [
    HumanMessage("I'm Alice from StartupX. We have a basic subscription."),
    HumanMessage("I prefer email communication and I'm interested in API features.")
]

# TODO: Extract initial profile
system_msg = "Extract customer profile from the following conversation:"
initial_result = # YOUR CODE HERE

print("Initial profile created:")
initial_profile = initial_result["responses"][0]
print(initial_profile.model_dump())

In [ ]:
# TODO: Update the profile with new information
update_conversation = [
    HumanMessage("We just upgraded to premium subscription!"),
    HumanMessage("Also, you can reach me at alice@startupx.io")
]

# TODO: Prepare existing profile data for Trustcall
existing_profile_data = {"CustomerProfile": initial_profile.model_dump()}

# TODO: Update profile using Trustcall
system_msg = "Update the customer profile with new information from the conversation:"
update_result = # YOUR CODE HERE - invoke extractor with existing data

print("Updated profile:")
updated_profile = update_result["responses"][0]
print(updated_profile.model_dump())

print("\nComparison:")
print(f"Subscription tier: {initial_profile.subscription_tier} → {updated_profile.subscription_tier}")
print(f"Email: {initial_profile.email} → {updated_profile.email}")

## Part 4: Collection-Based Memory

### Exercise 4.1: Learning Progress Tracker

Build a system that tracks a student's learning progress using collection-based memory.

In [ ]:
from datetime import datetime
from typing import Literal

# TODO: Define a LearningEntry schema
class LearningEntry(BaseModel):
    """Individual learning progress entry"""
    # TODO: Add fields for:
    # - topic: str (what was learned)
    # - skill_level: Literal["beginner", "intermediate", "advanced"]
    # - confidence: int (1-10 scale)
    # - notes: str (additional details)
    # - practice_needed: bool (whether more practice is needed)
    pass  # Replace with your implementation

# TODO: Test the schema
sample_entry = LearningEntry(
    topic="Python loops",
    skill_level="intermediate", 
    confidence=7,
    notes="Comfortable with for loops, need practice with nested loops",
    practice_needed=True
)

print(f"Sample learning entry: {sample_entry.model_dump()}")

In [ ]:
# TODO: Create a Trustcall extractor for learning entries
learning_extractor = create_extractor(
    model,
    tools=[LearningEntry],
    tool_choice="LearningEntry",
    enable_inserts=True  # Allow adding new entries
)

# TODO: Build a learning tracker chatbot
def learning_tracker(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Learning progress chatbot"""
    
    # TODO: Get user_id and retrieve learning entries
    user_id = # YOUR CODE HERE
    namespace = # YOUR CODE HERE
    entries = # YOUR CODE HERE
    
    # TODO: Format learning progress for system message
    progress_summary = "\n".join(f"- {entry.value}" for entry in entries) if entries else "No learning progress recorded yet."
    
    system_msg = f"""
    You are a learning progress assistant. You help track what the user has learned.
    
    Current learning progress:
    {progress_summary}
    
    Provide encouragement and suggest next steps based on their progress.
    """
    
    # TODO: Generate response
    response = # YOUR CODE HERE
    return {"messages": response}

def update_learning_progress(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Update learning progress based on conversation"""
    
    # TODO: Get user_id and namespace
    user_id = # YOUR CODE HERE
    namespace = # YOUR CODE HERE
    
    # TODO: Get existing entries for updates
    existing_entries = store.search(namespace)
    tool_name = "LearningEntry"
    existing_data = [(entry.key, tool_name, entry.value) for entry in existing_entries] if existing_entries else None
    
    # TODO: Extract learning updates
    system_msg = "Extract learning progress from the conversation. Create new entries or update existing ones:"
    result = # YOUR CODE HERE
    
    # TODO: Save updated entries
    for response, metadata in zip(result["responses"], result["response_metadata"]):
        key = metadata.get("json_doc_id", str(uuid.uuid4()))
        store.put(namespace, key, response.model_dump())

print("Learning tracker functions defined!")

In [ ]:
# TODO: Build and test the learning tracker
builder = StateGraph(MessagesState)
builder.add_node("learning_tracker", learning_tracker)
builder.add_node("update_learning_progress", update_learning_progress)
builder.add_edge(START, "learning_tracker")
builder.add_edge("learning_tracker", "update_learning_progress")
builder.add_edge("update_learning_progress", END)

learning_store = InMemoryStore()
learning_memory = MemorySaver()
learning_graph = builder.compile(checkpointer=learning_memory, store=learning_store)

# Test the learning tracker
config = {"configurable": {"thread_id": "learn_1", "user_id": "student_alex"}}
messages = [HumanMessage("I just learned about Python dictionaries! I understand the basics but I'm still confused about nested dictionaries.")]

for chunk in learning_graph.stream({"messages": messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# Continue tracking progress
messages = [HumanMessage("Today I practiced with nested dictionaries and I'm feeling much more confident now! I think I'm ready to learn about dictionary comprehensions.")]

for chunk in learning_graph.stream({"messages": messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# TODO: View all learning entries
def display_learning_progress(store, user_id):
    """Display all learning progress for a user"""
    namespace = ("learning", user_id)
    entries = store.search(namespace)
    
    print(f"\nLearning Progress for {user_id}:")
    print("=" * 40)
    for i, entry in enumerate(entries, 1):
        data = entry.value
        print(f"{i}. Topic: {data['topic']}")
        print(f"   Skill Level: {data['skill_level']}")
        print(f"   Confidence: {data['confidence']}/10")
        print(f"   Notes: {data['notes']}")
        print(f"   Practice Needed: {data['practice_needed']}")
        print()

display_learning_progress(learning_store, "student_alex")

## Part 5: Memory Agent with Multiple Schema Types

### Exercise 5.1: Smart Home Assistant

Build an advanced agent that manages different types of memory for a smart home system: user preferences, device status, and automation rules.

In [ ]:
from typing import Literal, Union
from datetime import time

# TODO: Define schemas for different memory types

class UserPreferences(BaseModel):
    """Smart home user preferences"""
    # TODO: Add fields for:
    # - preferred_temperature: Optional[int]
    # - wake_up_time: Optional[time]
    # - bedtime: Optional[time] 
    # - preferred_lighting: List[str]
    # - music_preferences: List[str]
    # - security_level: Literal["low", "medium", "high"]
    pass

class DeviceStatus(BaseModel):
    """Status of smart home devices"""
    # TODO: Add fields for:
    # - device_name: str
    # - device_type: Literal["light", "thermostat", "speaker", "camera", "lock"]
    # - location: str (room name)
    # - current_state: str (on/off, temperature, volume, etc.)
    # - last_updated: Optional[datetime]
    # - needs_attention: bool
    pass

class AutomationRule(BaseModel):
    """Automation rules for smart home"""
    # TODO: Add fields for:
    # - rule_name: str
    # - trigger: str (what triggers this rule)
    # - action: str (what action to take)
    # - active: bool
    # - schedule: Optional[str] (when to run)
    pass

# TODO: Define memory update decision schema
class UpdateMemory(BaseModel):
    """Decision on what memory to update"""
    # TODO: Add field:
    # - memory_type: Literal["preferences", "devices", "automation"]
    pass

print("Smart home schemas defined!")

In [ ]:
# TODO: Create extractors for each schema type
preferences_extractor = # YOUR CODE HERE
devices_extractor = # YOUR CODE HERE  
automation_extractor = # YOUR CODE HERE

# TODO: Define the smart home assistant system message
SMART_HOME_SYSTEM_MESSAGE = """
You are a smart home assistant that manages user preferences, device status, and automation rules.

Current User Preferences:
{preferences}

Current Device Status:
{devices}

Current Automation Rules:
{automation}

Instructions:
1. Analyze user messages for smart home-related updates
2. Use UpdateMemory tool to specify what type of memory to update
3. Provide helpful responses about the smart home system

When to update each memory type:
- preferences: User mentions temperature preferences, schedules, lighting preferences, etc.
- devices: User reports device status, issues, or changes
- automation: User wants to create, modify, or remove automation rules
"""

print("Smart home assistant configured!")

In [ ]:
# TODO: Implement smart home assistant functions

def smart_home_assistant(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Main smart home assistant function"""
    
    user_id = config["configurable"]["user_id"]
    
    # TODO: Retrieve all memory types
    # Preferences
    prefs_namespace = # YOUR CODE HERE
    prefs_memory = store.get(prefs_namespace, "user_preferences")
    preferences = # YOUR CODE HERE - handle existing vs empty
    
    # Devices  
    devices_namespace = # YOUR CODE HERE
    device_entries = # YOUR CODE HERE
    devices = "\n".join(f"- {entry.value}" for entry in device_entries) if device_entries else "No devices registered"
    
    # Automation
    automation_namespace = # YOUR CODE HERE  
    automation_entries = # YOUR CODE HERE
    automation = "\n".join(f"- {entry.value}" for entry in automation_entries) if automation_entries else "No automation rules set"
    
    # TODO: Create system message and get response
    system_msg = SMART_HOME_SYSTEM_MESSAGE.format(
        preferences=preferences,
        devices=devices, 
        automation=automation
    )
    
    response = model.bind_tools([UpdateMemory]).invoke(
        [SystemMessage(content=system_msg)] + state["messages"]
    )
    
    return {"messages": [response]}

# TODO: Implement memory update functions for each type
def update_preferences(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Update user preferences"""
    user_id = config["configurable"]["user_id"]
    namespace = ("preferences", user_id)
    
    # TODO: Handle existing preferences and update with Trustcall
    # YOUR CODE HERE
    
    # Return tool response
    tool_calls = state['messages'][-1].tool_calls
    return {"messages": [{"role": "tool", "content": "Preferences updated", "tool_call_id": tool_calls[0]['id']}]}

def update_devices(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Update device status"""
    # TODO: Similar to update_preferences but for devices
    # YOUR CODE HERE
    
    tool_calls = state['messages'][-1].tool_calls
    return {"messages": [{"role": "tool", "content": "Device status updated", "tool_call_id": tool_calls[0]['id']}]}

def update_automation(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """Update automation rules"""
    # TODO: Similar to other update functions but for automation
    # YOUR CODE HERE
    
    tool_calls = state['messages'][-1].tool_calls
    return {"messages": [{"role": "tool", "content": "Automation rules updated", "tool_call_id": tool_calls[0]['id']}]}

print("Smart home functions implemented!")

In [ ]:
# TODO: Create routing function and build the graph
def route_memory_update(state: MessagesState) -> Literal["update_preferences", "update_devices", "update_automation", END]:
    """Route to appropriate memory update function"""
    message = state['messages'][-1]
    
    if not message.tool_calls:
        return END
    
    memory_type = message.tool_calls[0]['args']['memory_type']
    
    # TODO: Return appropriate route based on memory_type
    # YOUR CODE HERE

# TODO: Build the graph
builder = StateGraph(MessagesState)

# TODO: Add all nodes and edges
# YOUR CODE HERE

# TODO: Compile the graph
smart_home_store = InMemoryStore()
smart_home_memory = MemorySaver()
smart_home_graph = # YOUR CODE HERE

print("Smart home assistant graph ready!")

In [ ]:
# Test the smart home assistant
config = {"configurable": {"thread_id": "home_1", "user_id": "homeowner_1"}}

# Test preferences update
messages = [HumanMessage("I prefer to wake up at 7 AM and I like the temperature at 72 degrees. Also, I prefer dim lighting in the evening.")]

for chunk in smart_home_graph.stream({"messages": messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# Test device status update
messages = [HumanMessage("The living room light is acting up - it keeps flickering. Also, the bedroom thermostat is currently set to 70 degrees.")]

for chunk in smart_home_graph.stream({"messages": messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
# Test automation rule creation
messages = [HumanMessage("I want to create an automation rule: turn on the front porch light at sunset and turn it off at 11 PM every day.")]

for chunk in smart_home_graph.stream({"messages": messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

## Part 6: Integration Challenge

### Exercise 6.1: Multi-User Memory System

**Challenge**: Create a family assistant that manages different types of memory for multiple family members, with shared household information and individual preferences.

In [ ]:
# TODO: Design schemas for:
# 1. Individual family member profiles
# 2. Shared household tasks/calendar
# 3. Family preferences (shared settings)
# 4. Individual preferences (personal settings)

class FamilyMember(BaseModel):
    """Individual family member profile"""
    # TODO: Design appropriate fields
    pass

class HouseholdTask(BaseModel):
    """Shared household tasks"""
    # TODO: Design appropriate fields  
    pass

class FamilySettings(BaseModel):
    """Shared family settings"""
    # TODO: Design appropriate fields
    pass

# TODO: Implement a family assistant that:
# 1. Identifies which family member is speaking
# 2. Updates appropriate individual or shared memory
# 3. Provides personalized responses based on family member
# 4. Manages conflicts between individual and shared preferences

print("Design your family assistant implementation here!")
print("Consider how to handle:")
print("- User identification and authentication")
print("- Shared vs individual memory spaces")
print("- Privacy between family members")
print("- Conflict resolution for shared resources")

## Key Takeaways

After completing these exercises, you should understand:

### When to Use Each Memory Pattern:

1. **Simple Key-Value Store**: For basic fact storage and retrieval
2. **Profile Schema**: For maintaining a single, evolving user profile
3. **Collection Schema**: For storing multiple independent but related items
4. **Multi-Schema Agent**: For complex systems requiring different types of memory

### Memory Design Principles:

1. **Namespace Organization**: Use clear, hierarchical namespaces
2. **Schema Evolution**: Design schemas that can grow and change
3. **Update Efficiency**: Use Trustcall for efficient partial updates
4. **Memory Scope**: Consider whether memory should be individual or shared

### Best Practices:

1. Start with simple patterns and evolve to complex ones
2. Use structured schemas for consistency and validation
3. Implement selective memory updating for efficiency
4. Design for both short-term and long-term memory needs
5. Consider privacy and access control in multi-user systems

### Next Steps:

- Experiment with different memory patterns in your own projects
- Explore persistent storage backends beyond InMemoryStore
- Implement memory compression and archiving for long-term systems
- Consider memory synchronization across distributed systems